# Data Preparation - accessing quality and cleaning data

## Load and and Present Data 

In [58]:
import sklearn
print(sklearn.__file__)  # Should print 1.3.1 or similar


None


In [61]:
import pandas as pd
import numpy as np
#from sklearn.impute import SimpleImputer
import plotly.express as px
#from scipy import stats

# Load the data set 
df = pd.read_csv('../data/raw_data/Children-and-young-ppl-asthma-organisational-audit-2019-20-Data.csv')
#Pivot Table to describe the dataset
print("Original Data")
df.info()
#Returning how many rows in the data frame 
print("\n The amount of organisations in the raw data set is",df.count().values[0],".") 

Original Data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Columns: 156 entries, orgcode to q8_2hasyourcomm_iatricasthmacare
dtypes: object(156)
memory usage: 173.2+ KB

 The amount of organisations in the raw data set is 142 .


## Data Quality Check and Cleaning - Irrelevant Columns

In [55]:
#Dropping irrelevant columns 
#Dropping columns with contain the following keywords which are irrelevant for this data analysis
previous_col_size= len(df.columns)
valsToRemove = ["WTE","q8","q7","q6","q5","q4","q3","othernotlisted","cat"]
listOfColumns = []

for column in df.columns:
    # converting data type for integer and float to numeric values 
    try:
        #+5 IS BECAUSE THE FIRST 5 COLUMNS ARE KNOWN TO BE NON-NUMERICAL
        df[column + 5] = pd.to_numeric(df[column], errors='coerce')  # Convert to numeric if possible
    except:
        pass
    
    if any(sub in column for sub in valsToRemove):
        listOfColumns.append(column)
    column_name = df[column].astype(str)
    wteString = column_name.str.contains("|".join(valsToRemove), case=False, na=False)
    if wteString.any():
        listOfColumns.append(column) 
df = df.drop(columns = listOfColumns)
succeeding_col_size = len(df.columns)
end_col_size = previous_col_size - succeeding_col_size
print(df)
print("There were", end_col_size,"irrelevant columns in the raw data set")

    orgcode                          description  \
0       ADD               Addenbrooke's Hospital   
1       AEI        Royal Albert Edward Infirmary   
2       AIR            Airedale General Hospital   
3       ALC  Royal Alexandra Children's Hospital   
4       BAR   Barnsley District General Hospital   
..      ...                                  ...   
137     WSH                West Suffolk Hospital   
138     WYB           Withybush General Hospital   
139     WYT                 Wythenshawe Hospital   
140     YDH               York District Hospital   
141     YEO             Yeovil District Hospital   

                                                 trust  country emergency_adm  \
0    Cambridge University Hospitals NHS Foundation ...  England           NaN   
1    Wrightington, Wigan and Leigh NHS Foundation T...  England           400   
2                        Airedale NHS Foundation Trust  England           110   
3    Brighton and Sussex University Hospitals NHS T

## Data Cleaning - Missing Values 

In [31]:
# Handling the different types of na values "-, n/a , null"
df = df.replace(['-','1-7'],np.nan)

#Number of missing values in whole dataset
total_missing_val= df.isna().sum().sum()

# How many NA values are in a row
print("EMPTY ROWS VALUES")
print(df.isna().sum(axis=1))

# How many NA values are in a column
print("\nEMPTY COLUMNS VALUES")
print(df.isna().sum(axis=0))

# Separate numeric and string columns
numeric_col = [col for col in df.columns if df[col].dtypes in ['int64', 'float64']]
string_columns = [col for col in df.columns if df[col].dtypes == 'object']

# Missing value analysis - EDA 
figure = px.imshow(df.isna(), color_continuous_scale="pinkyl",width=1000,height=600)
figure.update_layout(title="Missing Value Analysis on Dataset",margin=dict(t=50, b=50, l=50, r=50),)
figure.show()

# Filling NA values using simple impute and mode(most frequent)

# Impute numeric columns with mean
if numeric_col:
    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_col] = numeric_imputer.fit_transform(df[numeric_col])

# Impute string columns with the most frequent value
if string_columns:
    string_imputer = SimpleImputer(strategy='most_frequent')
    df[string_columns] = string_imputer.fit_transform(df[string_columns])

print("\n \n")    
print(df)
print("Total missing values that were replaced by SimpleImputer", total_missing_val )

# After
# How many NA values are in a row
print("EMPTY ROWS VALUES")
print(df.isna().sum(axis=1))

# How many NA values are in a column
print("\nEMPTY COLUMNS VALUES")
print(df.isna().sum(axis=0))


EMPTY ROWS VALUES
0      39
1       1
2       3
3       3
4       2
       ..
137     1
138    16
139     1
140     1
141     3
Length: 142, dtype: int64

EMPTY COLUMNS VALUES
orgcode                              0
description                          0
trust                                0
country                              0
emergency_adm                       28
                                    ..
asthmanursespecialistunfilledwte    11
nurseconsultant_nurseunfilledwte    12
specialistrespi_rapisunfilledwte    15
paediatricpsych_ogistunfilledwte    13
paediatricpharmacistunfilledwte     14
Length: 63, dtype: int64


NameError: name 'SimpleImputer' is not defined

## Data Quality Check and Cleaning - Duplicate Values

In [14]:
# Duplicate values
has_duplicates = df.duplicated().sum()
print("The number of duplicate organisations are ",has_duplicates) 

The number of duplicate organisations are  0


## Data Quality Check - Data Consistency

In [15]:
# Returns data type for the df       
print(df.dtypes)

orgcode                             object
description                         object
trust                               object
country                             object
emergency_adm                       object
                                     ...  
asthmanursespecialistunfilledwte    object
nurseconsultant_nurseunfilledwte    object
specialistrespi_rapisunfilledwte    object
paediatricpsych_ogistunfilledwte    object
paediatricpharmacistunfilledwte     object
Length: 63, dtype: object


## Data Cleaning - Data Usability

In [16]:
# Changing data frame column names
col_names = ["Hospital Code","Hospital Name","Trust Name","Country","Emergency Medical Admissions","Emergency Repository Admissions","Emergency Asthma Admissions", "Paediatric Admissions", "Repository Admissions", "Paediatric Asthma Admissions Per 1000 Admissions","Total Paediatric Beds For Asthma Patients","Paediatric HDU Present?", "Total Paediatric HDU Beds","Paediatric ICU Present?","Total Paediatric ICU Beds", "FY1/2 - Total Repository Staff Posts","FY1/2 - Staff Posts Per 100 Repository Paediatric Admissions", "FY1/2 - Staff Posts Per 100 Asthma Paediatric Admissions", "FY1/2 -Filled Respiratory Staff Posts","ST1/2 - Total Repository Staff Posts","ST1/2 - Staff Posts Per 100 Repository Paediatric Admissions","ST1/2 - Staff Posts Per 100 Asthma Paediatric Admissions", "ST1/2 -Filled Respiratory Staff Posts","ST3+ - Total Repository Staff Posts","ST3+ - Staff Posts Per 100 Repository Paediatric Admissions","ST3+ - Staff Posts Per 100 Asthma Paediatric Admissions", "ST3+ -Filled Respiratory Staff Posts","Repository Team Paediatric Consultant Filled Staff Post"," Paediatric Consultant Repository Admissions per 100 Admissions","Paediatric Consultant Asthma Admissions per 100 Admissions","Repository Team Paediatric Repository Consultant Filled Staff Post"," Paediatric Repository Consultant Repository Admissions per 100 Admissions","Paediatric Repository Consultant Asthma Admissions per 100 Admissions","Repository Team Associate Specialist Filled Staff Post"," Associate Specialist Repository Admissions per 100 Admissions","Associate Specialist Asthma Admissions per 100 Admissions","Repository Team Staff Grade Filled Staff Post"," Staff Grade Repository Admissions per 100 Admissions","Staff Grade Asthma Admissions per 100 Admissions","Repository Team Nurse Consultant Filled Staff Post"," Nurse Consultant Repository Admissions per 100 Admissions","Nurse Consultant Asthma Admissions per 100 Admissions", "Repository Team Specialist Repository Filled Staff Post"," Specialist Repository Repository Admissions per 100 Admissions"," Specialist Repository Asthma Admissions per 100 Admissions","Repository Team Paediatric Psychologist Filled Staff Post"," Paediatric Psychologist Repository Admissions per 100 Admissions","Paediatric Psychologist Asthma Admissions per 100 Admissions","Repository Team Paediatric Pharmacist Filled Staff Post"," Paediatric Pharmacist Repository Admissions per 100 Admissions","Paediatric Pharmacist Asthma Admissions per 100 Admissions", "FY1/2 - Unfilled Staff Posts in Repository Team","ST1/2 - Unfilled Staff Posts in Repository Team","ST3+ - Unfilled Staff Posts in Repository Team", "Unfilled Paediatric Consultant Staff Posts", "Unfilled Paediatric Repository Consultant Staff Posts","Unfilled Associate Specialist Staff Posts","Unfilled Staff Grade Staff Posts", "Unfilled Asthma Nurse Consultant Staff Posts","Unfilled  Nurse Consultant Staff Posts", "Unfilled Nurse Specialist Repository Staff Posts","Unfilled Paediatric Psychologist Staff Posts","Unfilled Paediatric Pharmacist Staff Posts"]
update_col_names = [f"{name}" for name in col_names]
df.columns = update_col_names
list_of_asthma = []

for name in col_names:
    #if the value contains the str "paediatric asthma" then add to list
    if "Asthma Admission" in name:
        list_of_asthma.append(name)
    else:
        continue
        
        
print(df)
print(list_of_asthma)

    Hospital Code                        Hospital Name  \
0             ADD               Addenbrooke's Hospital   
1             AEI        Royal Albert Edward Infirmary   
2             AIR            Airedale General Hospital   
3             ALC  Royal Alexandra Children's Hospital   
4             BAR   Barnsley District General Hospital   
..            ...                                  ...   
137           WSH                West Suffolk Hospital   
138           WYB           Withybush General Hospital   
139           WYT                 Wythenshawe Hospital   
140           YDH               York District Hospital   
141           YEO             Yeovil District Hospital   

                                            Trust Name  Country  \
0    Cambridge University Hospitals NHS Foundation ...  England   
1    Wrightington, Wigan and Leigh NHS Foundation T...  England   
2                        Airedale NHS Foundation Trust  England   
3    Brighton and Sussex University

## Data Cleaning - Removing Outliers 

In [17]:
# Filter numeric columns
numeric_cols = df.select_dtypes(include=['number'])

# Filter out columns with all NaN or constant values
valid_cols = numeric_cols.loc[:, numeric_cols.nunique(dropna=True) > 1]

#Diplaying outliers for each colum in the valid columns 
for col in valid_cols:
    z = np.abs(stats.zscore(df[col]))
    print(z)

threshold_z = 3

outlier_indices = np.where(z > threshold_z)[0]
no_outliers = df.drop(outlier_indices)
print("Original DataFrame Shape:", df.shape)
print("DataFrame Shape after Removing Outliers:", no_outliers.shape)
df

RecursionError: maximum recursion depth exceeded while calling a Python object

## Create Cleaned Data Frame

In [4]:
# Saving the cleaned data set into a new file 
df.to_csv("../data/cleaned_data/cleaned_dataset.csv", index=False)